In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

import sys
sys.path.append('../')
from config import Config, Shape
from plate import get_base, get_screw_positions, get_key_positions, make_plate
from case import get_basic_shape,  cut_aviator_connector_hole #, add_microcontrollerbox,

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


In [2]:
rkn = [3,5,8,5, 2]
rkn = [4,8,8,6]
config=Config(rkn, shape=Shape.LEAN, floorThickness=3)
kp = get_key_positions(config)

In [3]:
def add_microcontrollerbox(case, config):
        controller_y_offset = 5
        controllerBoxLength = config.controller.board_dimension_y - controller_y_offset
        controllerBoxWidth=config.controller.board_dimension_x + 5
        
        ctrBox = (case
                .edges('>Y and <Z')
                .workplane(centerOption="CenterOfBoundBox", invert=False)
                .box(controllerBoxWidth,controllerBoxLength, config.caseHeight, centered=[True,False,False],combine=False)
                )
        ctrBox.faces('<Z').tag("controllerBox")
        ctrBox.faces('>Z').tag("controllerBoxTop")

        case = case.union(ctrBox)
        return case

def make_case(config:Config) -> cq.Sketch:
    """TODO: add screw holes for microcontroller (check they are in right position!!)
            change fillet parameters
            add module for leds, encoder and aviator
            """
    
    
    case= get_basic_shape(config)
    #case.faces('<Z').tag('bottomface')

    # add box for microcontroller
    if config.controller is not None:
        case = add_microcontrollerbox(case, config)
    

    #fillet edges
    case = case.edges('|Z').fillet(config.edgeFillet)
    #case.edges('>Z').tag('outerTopEdge')


    # scoop out interior
    case= (case.faces(">Z or <Z").shell(-config.wallThickness, kind='intersection'))
    
    # set floor thickness 
    case = case.faces('<Z').wires('>X and >Y').toPending().extrude(config.floorThickness)
    
    #fillet bottom edge
    case = case.edges('<Z').fillet(3)



    # add plate screw holes 
    scr_hls = get_screw_positions(config)
    case=(case.faces('<Z').workplane(origin=(0,0))
          .pushPoints([(x,-y) for x,y in scr_hls])
          .cskHole(2.4, 4.8, 82, depth=None)    
          )

    # add controllerbox holes.
    # TODO - remove hardcoding on screw hole size
    if config.controller is not None:
        controller_y_offset = 5
        case = (case.workplaneFromTagged("controllerBox")
                .rect(config.controller.screw_hole_x,
                        config.controller.screw_hole_y- config.wallThickness - controller_y_offset - (config.controller.board_dimension_y -config.controller.screw_hole_y)*0.5, 
                        forConstruction=True ,centered=[True,True,True])
                .vertices()
                .cskHole(2.4, 4.8, 82, depth=None))
    
    
    #cut hole for aviator connector 
    case = cut_aviator_connector_hole(case, config)
    return case
make_case(config)



CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [ ]:
make_plate(config)

In [4]:
case=  make_case(config)


In [7]:

case.workplaneFromTagged("controllerBoxTop").box(10,10,2)

CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [11]:
case.faces(tag='controllerBoxTop')

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.00s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [ ]:
case

In [ ]:
ctrBox.faces()

In [ ]:
if True: 
    case= get_basic_shape(config)
    #case.faces('<Z').tag('bottomface')

    # add box for microcontroller
    #if config.controller is not None:
    #case = add_microcontrollerbox(case, config)

case.edges('<Y and <Z')

In [ ]:
case.faces(tag='controllerBoxTop')